In [1]:
# Load the extension that allows us to compile CUDA code in python notebooks
# Documentation is here: https://nvcc4jupyter.readthedocs.io/en/latest/
!pip install git+https://github.com/andreinechaev/nvcc4jupyter.git
%load_ext nvcc4jupyter

  Cloning https://github.com/andreinechaev/nvcc4jupyter.git to c:\users\tinyr\appdata\local\temp\pip-req-build-fu_p04o3
  Resolved https://github.com/andreinechaev/nvcc4jupyter.git to commit 28f872a2f99a1b201bcd0db14fdbc5a496b9bfd7
  Installing build dependencies: started
  Installing build dependencies: finished with status 'done'
  Getting requirements to build wheel: started
  Getting requirements to build wheel: finished with status 'done'
  Preparing metadata (pyproject.toml): started
  Preparing metadata (pyproject.toml): finished with status 'done'
Source files will be saved in "C:\Users\tinyr\AppData\Local\Temp\tmpeb22z1tg".


  Running command git clone --filter=blob:none --quiet https://github.com/andreinechaev/nvcc4jupyter.git 'C:\Users\tinyr\AppData\Local\Temp\pip-req-build-fu_p04o3'

[notice] A new release of pip is available: 24.0 -> 24.2
[notice] To update, run: python.exe -m pip install --upgrade pip


In [2]:
%%cuda_group_save -g "source" -n "data_types.h"
/**
 * A collection of commonly used data types throughout this project.
 */
#pragma once

#include <stdint.h> // uint32_t

using element_t = uint32_t;

In [3]:
%%cuda_group_save -g "source" -n "cuda_common.h"
/**
 * Standard macros that can be useful for error checking.
 * https://docs.nvidia.com/cuda/cuda-runtime-api/group__CUDART__ERROR.html
 */
#pragma once

#include <cuda.h>

#define CUDA_CALL(exp)                                       \
    do {                                                     \
        cudaError res = (exp);                               \
        if(res != cudaSuccess) {                             \
            printf("Error at %s:%d\n %s\n",                  \
                __FILE__,__LINE__, cudaGetErrorString(res)); \
           exit(EXIT_FAILURE);                               \
        }                                                    \
    } while(0)

#define CHECK_ERROR(msg)                                             \
    do {                                                             \
        cudaError_t err = cudaGetLastError();                        \
        if(cudaSuccess != err) {                                     \
            printf("Error (%s) at %s:%d\n %s\n",                     \
                (msg), __FILE__, __LINE__, cudaGetErrorString(err)); \
            exit(EXIT_FAILURE);                                      \
        }                                                            \
    } while (0)

In [4]:
%%cuda_group_save -g "source" -n "data_generator.h"
/**
 * Functions for generating random input data with a fixed seed
 */
#pragma once

#include <random>  // for std::mt19937, std::uniform_int_distribution
#include <vector>

#include "data_types.h"

namespace csc485b {
namespace a1 {

/**
 * Generates and returns a vector of random uniform data of a given length, n,
 * for any integral type. Input range will be [0, 2n].
 */
template < typename T >
std::vector< T > generate_uniform( std::size_t n )
{
    // for details of random number generation, see:
    // https://en.cppreference.com/w/cpp/numeric/random/uniform_int_distribution
    std::size_t random_seed = 20240916;  // use magic seed
    std::mt19937 rng( random_seed );     // use mersenne twister generator
    std::uniform_int_distribution<> distrib(0, 2 * n);

    std::vector< T > random_data( n ); // init array
    std::generate( std::begin( random_data )
                 , std::end  ( random_data )
                 , [ &rng, &distrib ](){ return static_cast< T >( distrib( rng ) ); });

    return random_data;
}

} // namespace a1
} // namespace csc485b

In [5]:
%%cuda_group_save -g "source" -n "algorithm_choices.h"
#pragma once

#include <vector>

#include "data_types.h"

namespace csc485b {
namespace a1 {
namespace cpu {

int run_cpu_baseline( std::vector< element_t > data, std::vector< element_t > &output, std::size_t switch_at, std::size_t n );

} // namespace cpu


namespace gpu {

int run_gpu_soln( std::vector< element_t > data, std::vector< element_t > &output, std::size_t switch_at, std::size_t n );

} // namespace gpu
} // namespace a1
} // namespace csc485b

In [6]:
%%cuda_group_save -g "source" -n "cpu_baseline.cu"
/**
 * CPU methods that the GPU should outperform.
 */

#include "algorithm_choices.h"

#include <algorithm> // std::sort()
#include <chrono>    // for timing
#include <iostream>  // std::cout, std::endl

namespace csc485b {
namespace a1      {
namespace cpu     {

/**
 * Simple solution that just sorts the whole array with a built-in sort
 * function and then resorts the last portion in the opposing order with
 * a second call to that same built-in sort function.
 */
void opposing_sort( element_t * data, std::size_t invert_at_pos, std::size_t num_elements )
{
    std::sort( data, data + num_elements, std::less< element_t >{} );
    std::sort( data + invert_at_pos, data + num_elements, std::greater< element_t >{} );
}

/**
 * Run the single-threaded CPU baseline that students are supposed to outperform
 * in order to obtain higher grades on this assignment. Times the execution and
 * prints to the standard output (e.g., the screen) that "wall time." Note that
 * the functions takes the input by value so as to not perturb the original data
 * in place.
 */
int run_cpu_baseline( std::vector< element_t > data, std::vector< element_t > &output, std::size_t switch_at, std::size_t n )
{
    auto const cpu_start = std::chrono::high_resolution_clock::now();
    opposing_sort( data.data(), switch_at, n );
    auto const cpu_end = std::chrono::high_resolution_clock::now();

    std::cout << "CPU Baseline time: "
              << std::chrono::duration_cast<std::chrono::nanoseconds>(cpu_end - cpu_start).count()
              << " ns" << std::endl;

    if(n <= 1 << 10) {
        for( auto const x : data ) std::cout << x << " "; std::cout << std::endl;
    }

    output = data;

    return std::chrono::duration_cast<std::chrono::nanoseconds>(cpu_end - cpu_start).count();
}

} // namespace cpu
} // namespace a1
} // namespace csc485b

In [7]:
%%cuda_group_save -g "source" -n "gpu_solution.cu"
/**
 * The file in which you will implement your GPU solutions!
 */

#include "algorithm_choices.h"

#include <chrono>    // for timing
#include <iostream>  // std::cout, std::endl
#include "cuda_common.h"
#include <cooperative_groups.h>
namespace cg = cooperative_groups;

namespace csc485b {
namespace a1      {
namespace gpu     {

/**
 * The CPU baseline benefits from warm caches because the data was generated on
 * the CPU. Run the data through the GPU once with some arbitrary logic to
 * ensure that the GPU cache is warm too and the comparison is more fair.
 */
__global__
void warm_the_gpu( element_t * data, std::size_t invert_at_pos, std::size_t num_elements )
{
    int const th_id = blockIdx.x * blockDim.x + threadIdx.x;

    // We know this will never be true, because of the data generator logic,
    // but I doubt that the compiler will figure it out. Thus every element
    // should be read, but none of them should be modified.
    if( th_id < num_elements && data[ th_id ] > num_elements * 100 )
    {
        ++data[ th_id ]; // should not be possible.
    }
}

/**
 * Your solution. Should match the CPU output.
 */
__global__
void opposing_sort( element_t * data, std::size_t invert_at_pos, std::size_t num_elements )
{
    int const th_id = blockIdx.x * blockDim.x + threadIdx.x;

    if( th_id < num_elements ) {
        // Pseudo code from https://en.wikipedia.org/wiki/Bitonic_sorter
        for(int outer = 2; outer <= num_elements; outer = outer << 1) {
            for(int inner = outer >> 1; inner > 0; inner = inner >> 1) {
                int pair = th_id ^ inner;
                int dir = th_id & outer;

                if(th_id < pair) {
                    if ( (dir == 0 && data[th_id] > data[pair]) || (dir != 0 && data[th_id] < data[pair]) ) {
                        int temp = data[th_id];
                        data[th_id] = data[pair];
                        data[pair] = temp;
                    }
                }

                __syncthreads();
            }
        }
        
        if( th_id >= 7 * num_elements / 8 ) {
          int pair = num_elements + invert_at_pos - th_id - 1;

          int temp = data[th_id];
          data[th_id] = data[pair];
          data[pair] = temp;
        }

        __syncthreads();

        return;
    }

}

__global__
void opposing_sort_coop( element_t * data, std::size_t invert_at_pos, std::size_t num_elements )
{
    cg::grid_group grid = cg::this_grid();

    int const th_id = blockIdx.x * blockDim.x + threadIdx.x;

    if( th_id < num_elements ) {
        // Pseudo code from https://en.wikipedia.org/wiki/Bitonic_sorter
        for(int outer = 2; outer <= num_elements; outer = outer << 1) {
            for(int inner = outer >> 1; inner > 0; inner = inner >> 1) {
                int pair = th_id ^ inner;
                int dir = th_id & outer;

                if(th_id < pair) {
                    if ( (dir == 0 && data[th_id] > data[pair]) || (dir != 0 && data[th_id] < data[pair]) ) {
                        int temp = data[th_id];
                        data[th_id] = data[pair];
                        data[pair] = temp;
                    }
                }

                cg::sync(grid);
            }
        }
        
        if( th_id >= 7 * num_elements / 8 ) {
          int pair = num_elements + invert_at_pos - th_id - 1;

          int temp = data[th_id];
          data[th_id] = data[pair];
          data[pair] = temp;
        }

        cg::sync(grid);

        return;
    }

}

__global__
void coop_sort( element_t * data, int start, int end, int direction ) {
    cg::grid_group grid = cg::this_grid();

    int const th_id = blockIdx.x * blockDim.x + threadIdx.x + start;

    if( th_id >= start && th_id < end ) {
        // Pseudo code from https://en.wikipedia.org/wiki/Bitonic_sorter
        for(int outer = 2; outer <= end - start; outer = outer << 1) {
            for(int inner = outer >> 1; inner > 0; inner = inner >> 1) {
                int pair = th_id ^ inner;
                int dir = (th_id & outer) - (outer * direction);

                if(th_id < pair) {
                    if ( (dir == 0 && data[th_id] > data[pair]) || (dir != 0 && data[th_id] < data[pair]) ) {
                        int temp = data[th_id];
                        data[th_id] = data[pair];
                        data[pair] = temp;
                    }
                }

                cg::sync(grid);
            }
        }
    }

    return;
}

__global__
void bitonic_merge_blocks( element_t * data, int outer, int inner, std::size_t num_elements ) {
    int const th_id = blockIdx.x * blockDim.x + threadIdx.x;

    if( th_id < num_elements ) {
        int pair = th_id ^ inner;
        int dir = th_id & outer;

        if(th_id < pair) {
            if ( (dir == 0 && data[th_id] > data[pair]) || (dir != 0 && data[th_id] < data[pair]) ) {
                int temp = data[th_id];
                data[th_id] = data[pair];
                data[pair] = temp;
            }
        }
    }

    return;
}

__global__
void reverse( element_t * data, std::size_t invert_at_pos, std::size_t num_elements ) {
    int const th_id = blockIdx.x * blockDim.x + threadIdx.x;

    if( th_id < num_elements ) {
        if( th_id >= 7 * num_elements / 8 ) {
            int pair = num_elements + invert_at_pos - th_id - 1;

            int temp = data[th_id];
            data[th_id] = data[pair];
            data[pair] = temp;
        }
    }
    
    return;
}

/**
 * Performs all the logic of allocating device vectors and copying host/input
 * vectors to the device. Times the opposing_sort() kernel with wall time,
 * but excludes set up and tear down costs such as mallocs, frees, and memcpies.
 */
int run_gpu_soln( std::vector< element_t > data, std::vector< element_t > &output, std::size_t switch_at, std::size_t n )
{
    // Kernel launch configurations. Feel free to change these.
    // This is set to maximise the size of a thread block on a T4, but it hasn't
    // been tuned. It's not known if this is optimal.
    std::size_t const threads_per_block = 1024;
    std::size_t const num_blocks =  ( n + threads_per_block - 1 ) / threads_per_block;

    // Allocate arrays on the device/GPU
    element_t * d_data;
    cudaMalloc( (void**) & d_data, sizeof( element_t ) * n );
    CHECK_ERROR("Allocating input array on device");

    // Copy the input from the host to the device/GPU
    cudaMemcpy( d_data, data.data(), sizeof( element_t ) * n, cudaMemcpyHostToDevice );
    CHECK_ERROR("Copying input array to device");

    int smemSize = threads_per_block * sizeof(int);

    cudaDeviceProp deviceProp;
    cudaGetDeviceProperties(&deviceProp, 0);

    int processors = deviceProp.multiProcessorCount;
    // processors = 2;

    // Warm the cache on the GPU for a more fair comparison
    warm_the_gpu<<< num_blocks, threads_per_block>>>( d_data, switch_at, n );

    // Time the execution of the kernel that you implemented
    auto const kernel_start = std::chrono::high_resolution_clock::now();

    if( num_blocks == 1) {
        opposing_sort<<< num_blocks, threads_per_block >>>( d_data, switch_at, n );

    } else if( num_blocks <= processors ) {
        void *kernelArgs[] = {
            (void *)&d_data, (void *)&switch_at, (void *)&n
        };

        dim3 dimBlock(threads_per_block, 1, 1);
        dim3 dimGrid(num_blocks, 1, 1);

        cudaLaunchCooperativeKernel((void *)opposing_sort_coop, dimGrid,
                                dimBlock, kernelArgs, smemSize, NULL);

    } else {
        // Round up to nearest power of two trick from https://graphics.stanford.edu/%7Eseander/bithacks.html#RoundUpPowerOf2
        uint32_t divisions = (num_blocks + processors - 1) / processors;
        divisions--;
        divisions |= divisions >> 1;
        divisions |= divisions >> 2;
        divisions |= divisions >> 4;
        divisions |= divisions >> 8;
        divisions |= divisions >> 16;
        divisions++;

        int total_threads = threads_per_block * num_blocks;

        // Create sorted subarrays
        int direction = 0;
        for(int i = 0; i < divisions; i++) {
            int start = (total_threads / divisions) * i;
            int end = start + (total_threads / divisions);

            void *kernelArgs[] = {
                (void *)&d_data, (void *)&start, (void *)&end, (void *)&direction
            };

            dim3 dimBlock(threads_per_block, 1, 1);
            dim3 dimGrid(num_blocks / divisions, 1, 1);

            cudaLaunchCooperativeKernel((void *)coop_sort, dimGrid,
                                    dimBlock, kernelArgs, smemSize, NULL);
        }

        // Merge/sort the subarrays
        for(int outer = 2 * n / divisions; outer < n; outer = outer << 1) {
            for(int inner = outer >> 1; inner > 0; inner = inner >> 1) {
                bitonic_merge_blocks<<< num_blocks, threads_per_block >>>( d_data, outer, inner, n );
            }
        }
        
        for(int inner = n >> 1; inner >= total_threads / divisions; inner = inner >> 1) {
            bitonic_merge_blocks<<< num_blocks, threads_per_block >>>( d_data, n, inner, n );
        }
        
        // Sort again
        direction = 0;
        for(int i = 0; i < divisions; i++) {
            int start = (total_threads / divisions) * i;
            int end = start + (total_threads / divisions);            

            void *kernelArgs[] = {
                (void *)&d_data, (void *)&start, (void *)&end, (void *)&direction
            };

            dim3 dimBlock(threads_per_block, 1, 1);
            dim3 dimGrid(num_blocks / divisions, 1, 1);

            cudaLaunchCooperativeKernel((void *)coop_sort, dimGrid,
                                    dimBlock, kernelArgs, smemSize, NULL);
            
            direction = 1 - direction;
        }
        
        reverse<<< num_blocks, threads_per_block >>>( d_data, switch_at, n );
    }

    auto const kernel_end = std::chrono::high_resolution_clock::now();
    CHECK_ERROR("Executing kernel on device");

    // After the timer ends, copy the result back, free the device vector,
    // and echo out the timings and the results.
    cudaMemcpy( data.data(), d_data, sizeof( element_t ) * n, cudaMemcpyDeviceToHost );
    CHECK_ERROR("Transferring result back to host");
    cudaFree( d_data );
    CHECK_ERROR("Freeing device memory");

    std::cout << "GPU Solution time: "
              << std::chrono::duration_cast<std::chrono::nanoseconds>(kernel_end - kernel_start).count()
              << " ns" << std::endl;

    if(n <= 1 << 10) {
        for( auto const x : data ) std::cout << x << " "; std::cout << std::endl;
    }
    
    output = data;

    return std::chrono::duration_cast<std::chrono::nanoseconds>(kernel_end - kernel_start).count();
}

} // namespace gpu
} // namespace a1
} // namespace csc485b

In [8]:
%%cuda_group_save -g "source" -n "main.cu"
/**
 * Driver for the benchmark comparison. Generates random data,
 * runs the CPU baseline, and then runs your code.
 */

#include <cstddef>  // std::size_t type
#include <iostream> // std::cout, std::endl
#include <vector>

#include "algorithm_choices.h"
#include "data_generator.h"
#include "data_types.h"
#include "cuda_common.h"

int main()
{
    std::size_t const n =  1 << 20;
    std::size_t const switch_at = 3 * ( n >> 2 ) ;

    auto data = csc485b::a1::generate_uniform< element_t >( n );

    std::vector<element_t> gpu_output;
    std::vector<element_t> cpu_output;

    std::cout << "List size: " << n << std::endl;

    int cpu_time = csc485b::a1::cpu::run_cpu_baseline( data, cpu_output, switch_at, n );
    int gpu_time = csc485b::a1::gpu::run_gpu_soln( data, gpu_output, switch_at, n );

    if(cpu_output == gpu_output) {
        std::cout << "Arrays match!" << std::endl;
    } else {
        std::cout << "Arrays DO NOT match!" << std::endl;

        for(int i=0; i<cpu_output.size(); i++) {
            if(cpu_output[i] != gpu_output[i]) {
                std::cout << "First mismatch at index " << i << "  " << cpu_output[i] << std::endl;
                
                break;
            }
        }
    }

    return EXIT_SUCCESS;
}

In [9]:
%cuda_group_run --group "source" --compiler-args "-O3 -g -std=c++20 -arch=sm_75"

List size: 1048576
CPU Baseline time: 61850000 ns
GPU Solution time: 309500 ns
Arrays match!

